# Curate datasets

Curating a dataset with LaminDB means three things:

1. Validate that the dataset matches a desired schema
2. If validation fails, standardize the dataset (e.g., by fixing typos, mapping synonyms) or update registries
3. Annotate the dataset by linking it against metadata entities so that it becomes queryable

In this guide we'll curate common data structures. Here is a [guide](/faq/curate-any) for the underlying low-level API.

Note: If you know either `pydantic` or `pandera`, here is an [FAQ](/faq/pydantic-pandera) that compares LaminDB with both of these tools.

In [ ]:
# pip install 'lamindb[bionty]'
!lamin init --storage ./test-curate --modules bionty

In [ ]:
import lamindb as ln

ln.track("MCeA3reqZG2e")

## DataFrame

### Allow a flexible schema

We'll be working with the mini immuno dataset:

In [ ]:
df = ln.core.datasets.mini_immuno.get_dataset1()
df

This is how we curate it in a script.

```{eval-rst}
.. literalinclude:: scripts/curate_dataframe_flexible.py
   :language: python
   :caption: curate_dataframe_flexible.py
```

Let's run the script.

In [ ]:
!python scripts/curate_dataframe_flexible.py

The script defined the following features & labels through {meth}`~lamindb.core.datasets.mini_immuno.define_features_labels`:

```{eval-rst}
.. literalinclude:: scripts/define_mini_immuno_features_labels.py
   :language: python
```

And the following schema through {meth}`~lamindb.examples.schemas.valid_features`:

```{eval-rst}
.. literalinclude:: scripts/define_valid_features.py
   :language: python
```

### Require a minimal set of columns

If we'd like to curate the dataframe with a minimal set of required columns, we can use the following schema.

```{eval-rst}
.. literalinclude:: scripts/define_mini_immuno_schema_flexible.py
   :language: python
```

If the dataframe lacks one of the required columns, we'll get a validation error.

```{eval-rst}
.. literalinclude:: scripts/curate_dataframe_minimal_errors.py
   :language: python
   :caption: curate_dataframe_minimal_errors.py
```

Let's run the script.

In [ ]:
!python scripts/curate_dataframe_minimal_errors.py

### Resolve synonyms and typos

Let's now look at the same dataset but assume there are synonyms and typos.

In [ ]:
df = ln.core.datasets.mini_immuno.get_dataset1(
    with_cell_type_synonym=True, with_cell_type_typo=True
)
df

Let's reuse the schema that defines a minimal set of columns we expect in the dataframe.

In [ ]:
schema = ln.core.datasets.mini_immuno.define_mini_immuno_schema_flexible()
schema.describe()

Create a curator object using the dataset & the schema.

In [ ]:
curator = ln.curators.DataFrameCurator(df, schema)

The {meth}`~lamindb.curators.core.Curator.validate` method validates that your dataset adheres to the criteria defined by the `schema`. It identifies which values are already validated (exist in the registries) and which are potentially problematic (do not yet exist in our registries).

In [ ]:
try:
    curator.validate()
except ln.errors.ValidationError as error:
    print(error)

In [ ]:
# check the non-validated terms
curator.cat.non_validated

For `cell_type`, we saw that "cerebral pyramidal neuron", "astrocytic glia" are not validated.

First, let's standardize synonym "astrocytic glia" as suggested

In [ ]:
curator.cat.standardize("cell_type_by_expert")

In [ ]:
# now we have only one non-validated cell type left
curator.cat.non_validated

For "CD8-pos alpha-beta T cell", let's understand which cell type in the public ontology might be the actual match.

In [ ]:
# to check the correct spelling of categories, pass `public=True` to get a lookup object from public ontologies
# use `lookup = curator.cat.lookup()` to get a lookup object of existing records in your instance
lookup = curator.cat.lookup(public=True)
lookup

In [ ]:
# here is an example for the "cell_type" column
cell_types = lookup["cell_type_by_expert"]
cell_types.cd8_positive_alpha_beta_t_cell

In [ ]:
# fix the cell type name
df["cell_type_by_expert"] = df["cell_type_by_expert"].cat.rename_categories(
    {"CD8-pos alpha-beta T cell": cell_types.cd8_positive_alpha_beta_t_cell.name}
)

For perturbation, we want to add the new values: "DMSO", "IFNG"

In [ ]:
# this adds perturbations that were _not_ validated
curator.cat.add_new_from("perturbation")

In [ ]:
# validate again
curator.validate()

Save a curated artifact.

In [ ]:
artifact = curator.save_artifact(key="examples/my_curated_dataset.parquet")

In [ ]:
artifact.describe()

## AnnData

`AnnData` like all other data structures that follow is a composite structure that stores different arrays in different `slots`.

### Allow a flexible schema

We can also allow a flexible schema for an `AnnData` and only require that it's indexed with Ensembl gene IDs.

```{eval-rst}
.. literalinclude:: scripts/curate_anndata_flexible.py
   :language: python
   :caption: curate_anndata_flexible.py
```

Let's run the script.

In [ ]:
!python scripts/curate_anndata_flexible.py

Under-the-hood, this used the following schema:

```{eval-rst}
.. literalinclude:: scripts/define_schema_anndata_ensembl_gene_ids_and_valid_features_in_obs.py
   :language: python
```

This schema tranposes the `var` DataFrame during curation, so that one validates and annotates the `var.T` schema, i.e., `[ENSG00000153563, ENSG00000010610, ENSG00000170458]`.
If one doesn't transpose, one would annotate with the schema of `var`, i.e., `[gene_symbol, gene_type]`.

```{eval-rst}
.. image:: https://lamin-site-assets.s3.amazonaws.com/.lamindb/gLyfToATM7WUzkWW0001.png
    :width: 800px
```

### Resolve typos

In [ ]:
import lamindb as ln

In [ ]:
adata = ln.core.datasets.mini_immuno.get_dataset1(
    with_gene_typo=True, with_cell_type_typo=True, otype="AnnData"
)
adata

In [ ]:
schema = ln.examples.schemas.anndata_ensembl_gene_ids_and_valid_features_in_obs()
schema.describe()

Check the slots of a schema:

In [ ]:
schema.slots

In [ ]:
curator = ln.curators.AnnDataCurator(adata, schema)
try:
    curator.validate()
except ln.errors.ValidationError as error:
    print(error)

As above, we leverage a lookup object with valid cell types to find the correct name.

In [ ]:
valid_cell_types = curator.slots["obs"].cat.lookup()["cell_type_by_expert"]
adata.obs["cell_type_by_expert"] = adata.obs[
    "cell_type_by_expert"
].cat.rename_categories(
    {"CD8-pos alpha-beta T cell": valid_cell_types.cd8_positive_alpha_beta_t_cell.name}
)

The validated `AnnData` can be subsequently saved as an {class}`~lamindb.Artifact`:

In [ ]:
adata.obs.columns

In [ ]:
curator.slots["var.T"].cat.add_new_from("columns")

In [ ]:
curator.validate()

In [ ]:
artifact = curator.save_artifact(key="examples/my_curated_anndata.h5ad")

Access the schema for each slot:

In [ ]:
artifact.features.slots

The saved artifact has been annotated with validated features and labels:

In [ ]:
artifact.describe()

## MuData

```{eval-rst}
.. literalinclude:: scripts/curate_mudata.py
   :language: python
   :caption: curate_mudata.py
```

In [ ]:
!python scripts/curate_mudata.py

## SpatialData

```{eval-rst}
.. literalinclude:: scripts/define_schema_spatialdata.py
   :language: python
   :caption: define_schema_spatialdata.py
```

In [ ]:
!python scripts/define_schema_spatialdata.py

```{eval-rst}
.. literalinclude:: scripts/curate_spatialdata.py
   :language: python
   :caption: curate_spatialdata.py
```

In [ ]:
!python scripts/curate_spatialdata.py

## TiledbsomaExperiment

```{eval-rst}
.. literalinclude:: scripts/curate_soma_experiment.py
   :language: python
   :caption: curate_soma_experiment.py
```

In [ ]:
!python scripts/curate_soma_experiment.py

## Other data structures

If you have other data structures, read: {doc}`/faq/curate-any`.

In [ ]:
!rm -rf ./test-curate
!rm -rf ./small_dataset.tiledbsoma
!lamin delete --force test-curate